In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import numpy as np

In [ ]:
tweets_sentiments = pd.read_csv('../data/interim/bitcoin_all_tweets_sentiments_20210101-20210930.csv', index_col = 0)

In [ ]:
tweets_sentiments.head()


In [ ]:
tweets_sentiments['dateTime']= pd.to_datetime(tweets_sentiments['dateTime'])


In [ ]:
tweets_sentiments = tweets_sentiments[['id', 'dateTime', 'prediction']]

tweets_sentiments.info()



In [ ]:
thirty_mins_span = tweets_sentiments.groupby('prediction').resample('30min', on='dateTime').count()

one_h_span = tweets_sentiments.groupby('prediction').resample('1H', on='dateTime').count()

two_h_span = tweets_sentiments.groupby('prediction').resample('2H', on='dateTime').count()

daily_span = tweets_sentiments.groupby('prediction').resample('D', on='dateTime').count()

fifteen_mins_span = tweets_sentiments.groupby('prediction').resample('15min', on='dateTime').count()

In [ ]:
thirty_mins_span = thirty_mins_span.unstack('prediction', fill_value=0)
thirty_mins_span = thirty_mins_span[('id')]

one_h_span = one_h_span.unstack('prediction', fill_value=0)
one_h_span = one_h_span[('id')]

two_h_span = two_h_span.unstack('prediction', fill_value=0)
two_h_span = two_h_span[('id')]

daily_span = daily_span.unstack('prediction', fill_value=0)
daily_span = daily_span[('id')]

fifteen_mins_span = fifteen_mins_span.unstack('prediction', fill_value=0)
fifteen_mins_span = fifteen_mins_span[('id')]


In [ ]:
two_h_span.index.name = None
two_h_span.columns.name = 'index'

#two_h_span.loc['2021-09-10':'2021-09-30']
daily_span.index.name = None
daily_span.columns.name = 'index'

#daily_span.loc['2021-09-10':'2021-09-30']

one_h_span.index.name = None
one_h_span.columns.name = 'index'

thirty_mins_span.index.name = None
thirty_mins_span.columns.name = 'index'


fifteen_mins_span.index.name = None
fifteen_mins_span.columns.name = 'index'



In [ ]:
def calc_percentage(df):
    df['total'] = df['negative'] + df['neutral'] + df['positive']
    df['pos_percent'] = df['positive'] / df['total'] * 100 
    df['neu_percent'] = df['neutral'] / df['total'] * 100
    df['neg_percent'] = df['negative'] / df['total'] * 100

calc_percentage(thirty_mins_span)
calc_percentage(one_h_span)
calc_percentage(two_h_span)
calc_percentage(daily_span)
calc_percentage(fifteen_mins_span)


In [ ]:
plot = daily_span

# Define the upper limit, lower limit, interval of Y axis and colors
y_LL = 0
y_UL = int(plot.iloc[:, 1:].max().max()*1.1)
#y_UL = 50000
y_interval = 4000
mycolors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange']    

# Draw Plot and Annotate
fig, ax = plt.subplots(1,1,figsize=(16, 9), dpi= 80)    

columns = plot.columns[:3]  
for i, column in enumerate(columns):    
    valt = plot[column].values
    plt.plot(plot.index.values, plot[column].values, lw=1.5, color=mycolors[i])    
    plt.text(plot.shape[0]+1, plot[column].values[-1], column, fontsize=14, color=mycolors[i])

# Draw Tick lines  
for y in range(y_LL, y_UL, y_interval):    
    plt.hlines(y, xmin=0, xmax=71, colors='black', alpha=0.3, linestyles="--", lw=0.5)

# Decorations    
plt.tick_params(axis="both", which="both", bottom=False, top=False,    
                labelbottom=True, left=False, right=False, labelleft=True)        

# Lighten borders
plt.gca().spines["top"].set_alpha(.3)
plt.gca().spines["bottom"].set_alpha(.3)
plt.gca().spines["right"].set_alpha(.3)
plt.gca().spines["left"].set_alpha(.3)

plt.title('a sample title', fontsize=22)
plt.yticks(range(y_LL, y_UL, y_interval), [str(y) for y in range(y_LL, y_UL, y_interval)], fontsize=12)    
plt.xticks(range(0, plot.shape[0], 24), plot.index.values[::24], horizontalalignment='left', fontsize=12)    


plt.ylim(y_LL, y_UL)    
plt.xlim(-2, 80)    
plt.show()

In [ ]:
plot = thirty_mins_span

x = plot.index.values
#pos = plot['positive']
#neu = plot['neutral']
#neg = plot['negative']

pos = plot['pos_percent']
neu = plot['neu_percent']
neg = plot['neg_percent']

mycolors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange']    

plt.plot(x,pos, color=mycolors[2])

plt.plot(x,neu, color=mycolors[1])
plt.plot(x,neg, color=mycolors[0])

plt.show()

In [ ]:
thirty_mins_span = thirty_mins_span.reset_index().rename(columns={'index':'datetime'})
one_h_span = one_h_span.reset_index().rename(columns={'index':'datetime'})
two_h_span = two_h_span.reset_index().rename(columns={'index':'datetime'})
daily_span = daily_span.reset_index().rename(columns={'index':'datetime'})
fifteen_mins_span = fifteen_mins_span.reset_index().rename(columns={'index':'datetime'})


In [ ]:
thirty_mins_span

In [ ]:
bitcoin_prices = pd.read_csv('../data/external/Bitstamp_BTCUSD_2021_minute_final.csv', header = 1)


In [ ]:
bitcoin_prices[-5:]


In [ ]:
bpd = bitcoin_prices[['date', 'open', 'Volume BTC']]
bpd['date'] = pd.to_datetime(bpd["date"], format="%Y-%m-%d %H:%M:%S", errors='coerce', utc=True)
#bpd['Date'] = bpd['SPdateTime'].dt.strftime('%Y-%m-%d')
bpd = bpd.set_index('date')
#bpd.reset_index()

bpd.info()
bpd.head()


In [ ]:
import datetime

def fetch_prices(df, span_mins):
    c_dts = df.datetime
    dts_1 = c_dts + datetime.timedelta(minutes=span_mins)
    dts_2 = c_dts + datetime.timedelta(minutes=(span_mins*2))
    dts_3 = c_dts + datetime.timedelta(minutes=(span_mins*3))
    dts_4 = c_dts + datetime.timedelta(minutes=(span_mins*4))

    df['cp'] = bpd.loc[c_dts].reset_index()['open']
    df['1spanp'] = bpd.loc[dts_1].reset_index()['open']
    df['2spanp'] = bpd.loc[dts_2].reset_index()['open']
    df['3spanp'] = bpd.loc[dts_3].reset_index()['open']
    df['4spanp'] = bpd.loc[dts_4].reset_index()['open']

fetch_prices(thirty_mins_span, 30)
fetch_prices(one_h_span, 60)
fetch_prices(two_h_span, 120)
#fetch_prices(two_h_span, 90)
fetch_prices(fifteen_mins_span, 15)




In [ ]:
def fetch_trade_vol(df, span_mins):
    c_dts = df.datetime
    vols = bpd.resample(str(span_mins) + 'min').sum()['Volume BTC']
    df['vol_btc'] = vols.loc[c_dts].reset_index()['Volume BTC']
fetch_trade_vol(thirty_mins_span, 30)
fetch_trade_vol(one_h_span, 60)
fetch_trade_vol(two_h_span, 120)
fetch_trade_vol(fifteen_mins_span, 15)



In [ ]:
thirty_mins_span


In [ ]:

thirty_diff = thirty_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
thirty_difs = thirty_diff.shift(-1)
thirty_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
thirty_mins_span_j = thirty_mins_span.join(thirty_difs)


one_h_diff = one_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
one_h_difs = one_h_diff.shift(-1)
one_h_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
one_h_span_j = one_h_span.join(one_h_difs)

two_h_diff = two_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
two_h_difs = two_h_diff.shift(-1)
two_h_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
two_h_span_j = two_h_span.join(two_h_difs)

fifteen_mins_diff = fifteen_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 2)
fifteen_mins_difs = fifteen_mins_diff.shift(-1)
fifteen_mins_difs.rename(columns={'pos_percent': 'pos_change', 'neu_percent': 'neu_change', 'neg_percent': 'neg_change'}, inplace=True)
fifteen_mins_span_j = fifteen_mins_span.join(fifteen_mins_difs)


In [ ]:
thirty_pdiff = thirty_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
thirty_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
thirty_mins_span_j = thirty_mins_span_j.join(thirty_pdiff)

one_h_pdiff = one_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
one_h_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
one_h_span_j = one_h_span_j.join(one_h_pdiff)

two_h_pdiff = two_h_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
two_h_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
two_h_span_j = two_h_span_j.join(two_h_pdiff)

fifteen_mins_pdiff = fifteen_mins_span[['pos_percent', 'neu_percent', 'neg_percent']].diff(periods = 1)
fifteen_mins_pdiff.rename(columns={'pos_percent': 'pos_pre_change', 'neu_percent': 'neu_pre_change', 'neg_percent': 'neg_pre_change'}, inplace=True)
fifteen_mins_span_j = fifteen_mins_span_j.join(fifteen_mins_pdiff)


In [ ]:
def assign_shift_target(df):
    df['shift1_up'] = np.where(df['2spanp'] >  df['1spanp'], 1, 0)
    df['shift2_up'] = np.where(df['3spanp'] >  df['2spanp'], 1, 0)
    df['shift3_up'] = np.where(df['4spanp'] >  df['3spanp'], 1, 0)

assign_shift_target(thirty_mins_span_j)
assign_shift_target(one_h_span_j)
assign_shift_target(two_h_span_j)
assign_shift_target(fifteen_mins_span_j)

In [ ]:
one_h_span_j

In [ ]:
thirty_mins_span_final = thirty_mins_span_j[['total', 'pos_percent', 'neu_percent', 'neg_percent', 'vol_btc', 'pos_change', 'neu_change', 'neg_change', 'pos_pre_change', 'neu_pre_change', 'neg_pre_change', 'shift1_up', 'shift2_up', 'shift3_up']]
one_h_span_final = one_h_span_j[['total', 'pos_percent', 'neu_percent', 'neg_percent', 'vol_btc', 'pos_change', 'neu_change', 'neg_change', 'pos_pre_change', 'neu_pre_change', 'neg_pre_change','shift1_up', 'shift2_up', 'shift3_up']]
two_h_span_final = two_h_span_j[['total', 'pos_percent', 'neu_percent', 'neg_percent', 'vol_btc', 'pos_change', 'neu_change', 'neg_change', 'pos_pre_change', 'neu_pre_change', 'neg_pre_change','shift1_up', 'shift2_up', 'shift3_up']]
fifteen_mins_span_final = fifteen_mins_span_j[['total', 'pos_percent', 'neu_percent', 'neg_percent', 'vol_btc', 'pos_change', 'neu_change', 'neg_change', 'pos_pre_change', 'neu_pre_change', 'neg_pre_change','shift1_up', 'shift2_up', 'shift3_up']]

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    TimeSeriesSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


In [ ]:
#thirty_mins_span_final.shape
#one_h_span_final.shape
#two_h_span.shape
fifteen_mins_span_final.head()

In [ ]:
n_train = 2000
thirty_train_df = thirty_mins_span_final[:2500]
thirty_test_df = thirty_mins_span_final[2500:]

one_h_train_df = one_h_span_final[:1600]
one_h_test_df = one_h_span_final[1600:]

two_h_train_df = two_h_span_final[:500]
two_h_test_df = two_h_span_final[500:]

fifteen_mins_train_df = fifteen_mins_span_final[:6000]
fifteen_mins_test_df = fifteen_mins_span_final[6000:]


In [ ]:
plt.figure(figsize=(10, 3))

plt.plot(thirty_train_df['pos_percent'], "b", label="train")
plt.plot(thirty_test_df['pos_percent'], "r", label="test")
plt.xticks(rotation="vertical")
plt.legend();

In [ ]:
numeric_features = [
    "pos_percent",
    "neu_percent",
    "neg_percent",
    "pos_change",
    "neu_change",
    "neg_change",
    "pos_pre_change",
    "neu_pre_change",
    "neg_pre_change",
    "vol_btc",
    "total"
]
categorical_features = [
    
]
drop_features = [
    "shift1_up",
    "shift2_up",
    "shift3_up",
  
]


In [ ]:
def preprocess_features(
    train_df,
    test_df,
    numeric_features,
    categorical_features,
    drop_features,
):

    all_features = set(numeric_features + categorical_features + drop_features)
    if set(train_df.columns) != all_features:
        print("Missing columns", set(train_df.columns) - all_features)
        print("Extra columns", all_features - set(train_df.columns))
        raise Exception("Columns do not match")

    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="median"), StandardScaler()
    )
    '''categorical_transformer = make_pipeline(
        SimpleImputer(strategy="constant", fill_value="?"),
        OneHotEncoder(handle_unknown="ignore", sparse=False),
    )'''

    preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        #(categorical_transformer, categorical_features),
        ("drop", drop_features),
    )
    preprocessor.fit(train_df)
    '''ohe_feature_names = (
        preprocessor.named_transformers_["pipeline-2"]
        .named_steps["onehotencoder"]
        .get_feature_names()
        .tolist()
    )'''
    #new_columns = numeric_features + ohe_feature_names
    new_columns = numeric_features
    X_train_enc = pd.DataFrame(
        preprocessor.transform(train_df), index=train_df.index, columns=new_columns
    )
    X_test_enc = pd.DataFrame(
        preprocessor.transform(test_df), index=test_df.index, columns=new_columns
    )

    y_train = train_df["shift2_up"]
    y_test = test_df["shift2_up"]

    return X_train_enc, y_train, X_test_enc, y_test, preprocessor

In [ ]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    fifteen_mins_train_df,
    fifteen_mins_test_df,
    numeric_features,
    categorical_features,
    drop_features,
)

In [ ]:
X_train_enc.head()

In [ ]:
def score_lr_print_coeff(preprocessor, train_df, y_train, test_df, y_test, X_train_enc):
    lr_pipe = make_pipeline(preprocessor, LogisticRegression(max_iter=1000))
    lr_pipe.fit(train_df, y_train)
    print("Train score: {:.2f}".format(lr_pipe.score(train_df, y_train)))
    print("Test score: {:.2f}".format(lr_pipe.score(test_df, y_test)))
    lr_coef = pd.DataFrame(
        data=lr_pipe.named_steps["logisticregression"].coef_.flatten(),
        index=X_train_enc.columns,
        columns=["Coef"],
    )
    return lr_coef.sort_values(by="Coef", ascending=False)

In [ ]:
# 30mins, 0930-0720, train 0.55, test 0.53, 1 shift
score_lr_print_coeff(preprocessor, fifteen_mins_train_df, y_train, fifteen_mins_test_df, y_test, X_train_enc)
